<img src="img/pandora2d_logo.png" width="500" height="500">

# Pandora2D : a coregistration framework

# Usage of step and ROI (Region of Interest) parameters in the configuration file

#### Imports and external functions

In [ ]:
import io
from pprint import pprint
from pathlib import Path
from IPython.display import Image, display
import numpy as np

In [ ]:
def plot_state_machine(pandora2d_machine):
    """
    Show the schemes of step of Pandora2D Machine
    """
    stream = io.BytesIO()
    try:
        pandora2d_machine.get_graph().draw(stream, prog='dot', format='png')
        display(Image(stream.getvalue()))
    except:
        print("It is not possible to show the graphic of the state machine. To solve it, please install graphviz on your system (apt-get install graphviz if operating in Linux) and install python package with pip insall graphviz")

In [ ]:
from snippets.utils import *

# Pandora2D execution options with state machine

#### Imports of pandora2d

In [ ]:
# Load pandora2d imports
from pandora2d import run 
from pandora2d.state_machine import Pandora2DMachine
from pandora2d.check_configuration import check_conf
from pandora2d.img_tools import get_roi_processing, create_datasets_from_inputs

#### Load and visualize input data 

Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "data/left.tif"
img_right_path = "data/right.tif"

Provide output directory to write results

In [ ]:
output_dir = Path.cwd() / "output"
# If necessary, create output dir
output_dir.mkdir(exist_ok=True,parents=True)

Convert input data to dataset

In [ ]:
input_config = {
    "left": {"img": img_left_path, "nodata": np.nan},
    "right": {"img": img_right_path, "nodata": np.nan},
    "col_disparity": [-2, 2],
    "row_disparity": [-2, 2],
}

In [ ]:
img_left, img_right = create_datasets_from_inputs(input_config=input_config)

Visualize input data

In [ ]:
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(1,2,1)
ax0.imshow(img_left["im"].data)
plt.title("Left image")
ax1 = fig.add_subplot(1,2,2)
ax1.imshow(img_right["im"].data)
plt.title("Right image")

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Define pipeline configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": img_left_path,
            "nodata": "NaN",
        },
        "right": {
            "img": img_right_path,
            "nodata": "NaN",
        },
        "col_disparity": [-3, 3],
        "row_disparity": [-3, 3],
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 7,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -9999
        },
        "refinement" : {
            "refinement_method" : "dichotomy",
            "iterations": 2,
            "filter": "sinc",
        }
    }
}

#### Check the user configuration

In [ ]:
cfg = check_conf(user_cfg, pandora2d_machine)
pprint(cfg)